<a href="https://colab.research.google.com/github/alexdibol/ML_FINANCE_TRAINING/blob/master/SEGUNDO_RESPALDO_DE_4_3_1_RNN_KERAS_AND_TB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RECURRENT NEURAL NETWORKS

## 1. MOTIVATION

Recurrent neural networks (RNN) are a class of neural networks that is powerful for modeling sequence data such as time series or natural language.

Schematically, a RNN layer uses a `for` loop to iterate over the timesteps of a sequence, while maintaining an internal state that encodes information about the timesteps it has seen so far.

The Keras RNN API is designed with a focus on:

- **Ease of use**: the built-in `tf.keras.layers.RNN`, `tf.keras.layers.LSTM`, `tf.keras.layers.GRU` layers enable you to quickly build recurrent models without having to make difficult configuration choices.
  
- **Ease of customization**: You can also define your own RNN cell layer (the inner part of the `for` loop) with custom behavior, and use it with the generic `tf.keras.layers.RNN` layer (the `for` loop itself). This allows you to quickly prototype different research ideas in a flexible way with minimal code.
  

## 2. A SIMPLE MODEL WITH AN EMBEDDING LAYER

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/word_embeddings.ipynb#scrollTo=SIXEk5ON5P7h

In [0]:
#!pip install tf-nightly

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x

except Exception:
    pass
import tensorflow as tf

# Load the TensorBoard notebook extension.
%load_ext tensorboard

from tensorflow.keras import layers
from datetime import datetime

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

### 2.1. GENERAL CONCEPT: INCLUDING THE EMBEDDING LAYER

When you create an Embedding layer, the weights for the embedding are randomly initialized (just like any other layer). During training, they are gradually adjusted via backpropagation. Once trained, the learned word embeddings will roughly encode similarities between words (as they were learned for the specific problem your model is trained on).

If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

For text or sequence problems, the Embedding layer takes a 2D tensor of integers, of shape `(samples, sequence_length)`, where each entry is a sequence of integers. It can embed sequences of variable lengths. You could feed into the embedding layer above batches with shapes `(32, 10)` (batch of 32 sequences of length 10) or `(64, 15)` (batch of 64 sequences of length 15).

The returned tensor has one more axis than the input, the embedding vectors are aligned along the new last axis. Pass it a `(2, 3)` input batch and the output is `(2, 3, N)`


## 2.2. LOADING THE DATA

In [0]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k', 
    split = (tfds.Split.TRAIN, tfds.Split.TEST), 
    with_info=True, as_supervised=True)

### 2.3. PREPARING THE DATA

Get the encoder (`tfds.features.text.SubwordTextEncoder`), and have a quick look at the vocabulary. 

The "\_" in the vocabulary represent spaces. Note how the vocabulary includes whole words (ending with "\_") and partial words which it can use to build larger words:

Movie reviews can be different lengths. We will use the padded_batch method to standardize the lengths of the reviews.

In [0]:
encoder = info.features['text'].encoder
#encoder.subwords[:20]

In [0]:
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))

In [0]:
type(train_batches)

## 3. SETTING UP THE MODEL

We will use the Keras Sequential API to define our model. In this case it is a "Continuous bag of words" style model.

Next the Embedding layer takes the integer-encoded vocabulary and looks up the embedding vector for each word-index. These vectors are learned as the model trains. The vectors add a dimension to the output array. The resulting dimensions are: (batch, sequence, embedding).

Next, a GlobalAveragePooling1D layer returns a fixed-length output vector for each example by averaging over the sequence dimension. This allows the model to handle input of variable length, in the simplest way possible.

This fixed-length output vector is piped through a fully-connected (Dense) layer with 16 hidden units.

The last layer is densely connected with a single output node. Using the sigmoid activation function, this value is a float between 0 and 1, representing a probability (or confidence level) that the review is positive.

Caution: This model doesn't use masking, so the zero-padding is used as part of the input, so the padding length may affect the output. To fix this, see the masking and padding guide.

In [0]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    verbose=0, # Suppress chatty output
    validation_data=test_batches, validation_steps=20)

## 4. PLOTTING THE RESULTS

With this approach our model reaches a validation accuracy of around 88% (note the model is overfitting, training accuracy is significantly higher).

In [0]:
import matplotlib.pyplot as plt

history_dict = history.history

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss=history_dict['loss']
val_loss=history_dict['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss, 'b', label='Training loss', color='magenta')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid()
plt.legend()
plt.show()

plt.figure(figsize=(8,6))
plt.plot(epochs, acc, 'b', label='Training acc', color='magenta')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.ylim((0.5,1))
plt.show()

## 5. PREPARE FOR EXTERNAL TENSORBOARD

In [0]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

In [0]:
import io
encoder = info.features['text'].encoder

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for num, word in enumerate(encoder.subwords):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

To visualize our embeddings we will upload them to the embedding projector.

Open the Embedding Projector (this can also run in a local TensorBoard instance).

- Click on "Load data".

- Upload the two files we created above: vecs.tsv and meta.tsv.

- The embeddings you have trained will now be displayed. You can search for words to find their closest neighbors. For example, try searching for "beautiful". You may see neighbors like "wonderful".

**Note: your results may be a bit different, depending on how weights were randomly initialized before training the embedding layer.**

`Note: experimentally, you may be able to produce more interpretable embeddings by using a simpler model. Try deleting the Dense(16) layer, retraining the model, and visualizing the embeddings again.`

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

USE THE EXTERNAL PROJECTOR

http://projector.tensorflow.org/

## 6.  THE 1-2-3 FOR USING THE BUILT IN TENSORBOARD

### STEP 1. LOADING THE PLUG-IN

In [0]:
import os
from tensorboard.plugins import projector

### STEP 2. CREATION OF DIRECTORY, WRITER AND CALLBACK FUNCTION

In [0]:
LOGDIR='content/logs/imdb-example/'
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=LOGDIR)
file_writer = tf.summary.create_file_writer(LOGDIR)

### STEP 3. COMPILATION AND EXECUTION OF THE MODEL

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=10,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback],
    validation_data=test_batches, validation_steps=20)

In [0]:
%tensorboard --logdir 'content/logs/imdb-example/'

## 7. THE 1-2-3 OF INCLUDING EMBEDDINGS WITH TENSORBOARD

In [0]:
#import os
#from tensorboard.plugins import projector

### STEP 1. SAVING THE LABELS: METDATA.TSV FILE

In [0]:
# Save Labels separately on a line-by-line manner.
with open(os.path.join(LOGDIR, 'metadata.tsv'), "w") as f:
  for subwords in encoder.subwords:
    f.write("{}\n".format(subwords))
  # Fill in the rest of the labels with "unknown"
  for unknown in range(1, encoder.vocab_size - len(encoder.subwords)):
    f.write("unknown #{}\n".format(unknown))

### STEP 2. RETRIEVE AND SAVE WEIGHTS OF THE FIRST (EMBEDDING) LAYER

In [0]:
# Save the weights we want to analyse as a variable. Note that the first
weights = tf.Variable(model.layers[0].get_weights()[0][1:])

### STEP 3. CREATE A CHECKPOINT AND CONFIGURE THE PROJECTOR

In [0]:
# Create a checkpoint from embedding, the filename and key are
# name of the tensor.
checkpoint = tf.train.Checkpoint(embedding=weights)
checkpoint.save(os.path.join(LOGDIR, "embedding.ckpt"))

In [0]:
# Set up config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()

# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings(LOGDIR, config)

In [0]:
%tensorboard --logdir 'content/logs/imdb-example/'

## 8. INCLUDING A RECURRENT LAYER IN THE NEURAL NETWORK

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/keras/rnn.ipynb

In [0]:
embedding_dim=16

model = keras.Sequential([
  layers.Embedding(encoder.vocab_size, embedding_dim),
  #layers.SimpleRNN(1),
  #layers.GlobalAveragePooling1D(),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

model.summary()

In [0]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_batches,
    epochs=3,
    verbose=0, # Suppress chatty output
    callbacks=[tensorboard_callback],
    validation_data=test_batches, validation_steps=20)

In [0]:
history